In [ ]:
logTransposeConst = 1

def notPresent(key, complexesFunc):
    amounts = complexesFunc[key]
    for num in amounts:
        if not num == 0:
            return False
    return True
    
def zeroFlux(key, dictionary):
    fluxes = dictionary[key]
    for num in fluxes:
        if not num == 0:
            return False
    return True


def createTuples(dictionary):
    result = []
    keys = dictionary.keys()
    for key in keys:
        mean = statistics.mean(dictionary[key])
        result.append(tuple((key, mean)))
    return result


def getZeroFluxReactions(reactionDictionary):
    reactions = reactionDictionary.keys()
    zeroFluxReactions = []

    for key in reactions:
        if zeroFlux(key, reactionDictionary):
            zeroFluxReactions.append(key)
            
    return zeroFluxReactions


def createHeatMapFluxes(reactionDictionary, mapHeight, annot_grid):
    reactions = reactionDictionary.keys()
    zeroFluxReactions = []
    nonZeroFlux = {}

    for key in reactions:
        if zeroFlux(key, reactionDictionary):
            zeroFluxReactions.append(key)
        else:
            res = [abs(ele) for ele in reactionDictionary[key]]
            nonZeroFlux[key] = res

    transformedData = {}
    for key in nonZeroFlux.keys():
        new = []
        old = nonZeroFlux[key]
        for val in old:
            new.append(math.log(val + logTransposeConst))
        transformedData[key] = new

    tuples = createTuples(transformedData)
    tuples.sort(key=lambda x: x[1])

    sortedReactions = {}
    for elem in tuples:
        key = elem[0]
        sortedReactions[key] = transformedData[key]

    df = pd.DataFrame.from_dict(sortedReactions, orient='index')
    df.index.name = 'Reactions'
    df.reset_index(inplace=False)
    
    revertedCounts = {}
    for key in sortedReactions.keys():
        temp = []
        for val in sortedReactions[key]:
            temp.append((math.e ** val) - 1)
        revertedCounts[key] = temp

    df2 = pd.DataFrame.from_dict(revertedCounts, orient='index')
    df2.index.name = 'Reactions'
    df2.reset_index(inplace=False)
    
    fig, ax = plt.subplots(figsize=(50, mapHeight))
    plt.xlabel("Seconds")
    if annot_grid:
        sns.heatmap(df, cmap="rocket_r", linewidths=.5, annot=df2) 
    else:
        sns.heatmap(df, cmap="rocket_r")
    return 

def createHeatMapComplexes(complexes, mapHeight, annot_grid):
    complexNames = complexes.keys()
    zeroQuantComplexes = []
    presentComplexes = {}

    for key in complexes:
        if notPresent(key, complexes):
            zeroQuantComplexes.append(key)
        else:
            presentComplexes[key] = complexes[key]

    transformedData = {}
    for key in presentComplexes.keys():
        new = []
        old = presentComplexes[key]
        for val in old:
            new.append(math.log(val + logTransposeConst))
        transformedData[key] = new

    tuples = createTuples(transformedData)
    tuples.sort(key=lambda x: x[1])

    sortedComplexes = {}
    for elem in tuples:
        key = elem[0]
        sortedComplexes[key] = transformedData[key]

    df = pd.DataFrame.from_dict(sortedComplexes, orient='index')
    df.index.name = 'Reactions'
    df.reset_index(inplace=False)

    fig, ax = plt.subplots(figsize=(50, mapHeight))
    plt.xlabel("Seconds")
    if annot_grid:
        sns.heatmap(df, cmap="rocket_r", linewidths=.5, annot=True)
    else:
        sns.heatmap(df, cmap="rocket_r")
    return 